In [2]:
# !wget https://f000.backblazeb2.com/file/malaya-model/pretrained/t5-base-social-media-2021-11-15.tar.gz
# !tar -zxf t5-base-social-media-2021-11-15.tar.gz
# !rm t5-base-social-media-2021-11-15.tar.gz
!ls t5-base-social-media

model.ckpt-1000000.data-00000-of-00002	model.ckpt-1000000.meta
model.ckpt-1000000.data-00001-of-00002	operative_config.gin
model.ckpt-1000000.index


In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [4]:
from transformers import T5Config, T5Model, load_tf_weights_in_t5

In [5]:
out = 't5-base'
os.makedirs(out, exist_ok=True)

In [6]:
!cat t5-base-social-media/operative_config.gin

import mesh_tensorflow.optimize
import mesh_tensorflow.transformer.dataset
import mesh_tensorflow.transformer.learning_rate_schedules
import mesh_tensorflow.transformer.t2t_vocabulary
import mesh_tensorflow.transformer.transformer_layers
import mesh_tensorflow.transformer.utils
import t5.data.sentencepiece_vocabulary
import t5.models.mesh_transformer

# Macros:
# ==============================================================================
d_ff = 3072
d_kv = 64
d_model = 768
dropout_rate = 0.1
init_checkpoint = 'gs://mesolitica-general/t5-base/model.ckpt-100000'
inputs_length = 1024
mean_noise_span_length = 3.0
MIXTURE_NAME = 'all_mix'
noise_density = 0.15
num_heads = 12
num_layers = 12

# Parameters for AdafactorOptimizer:
# ==============================================================================
AdafactorOptimizer.beta1 = 0.0
AdafactorOptimizer.clipping_threshold = 1.0
AdafactorOptimizer.decay_rate = None
AdafactorOptimizer.epsilon1 = 1e-30
AdafactorOptimizer.epsilon2 = 0.001


In [7]:
import tensorflow as tf
tf.train.list_variables('t5-base-social-media/model.ckpt-1000000')

[('decoder/block_000/layer_000/SelfAttention/k', [768, 768]),
 ('decoder/block_000/layer_000/SelfAttention/k_slot_vc', [768]),
 ('decoder/block_000/layer_000/SelfAttention/k_slot_vr', [768]),
 ('decoder/block_000/layer_000/SelfAttention/o', [768, 768]),
 ('decoder/block_000/layer_000/SelfAttention/o_slot_vc', [768]),
 ('decoder/block_000/layer_000/SelfAttention/o_slot_vr', [768]),
 ('decoder/block_000/layer_000/SelfAttention/q', [768, 768]),
 ('decoder/block_000/layer_000/SelfAttention/q_slot_vc', [768]),
 ('decoder/block_000/layer_000/SelfAttention/q_slot_vr', [768]),
 ('decoder/block_000/layer_000/SelfAttention/relative_attention_bias',
  [12, 32]),
 ('decoder/block_000/layer_000/SelfAttention/relative_attention_bias_slot_v',
  [12, 32]),
 ('decoder/block_000/layer_000/SelfAttention/v', [768, 768]),
 ('decoder/block_000/layer_000/SelfAttention/v_slot_vc', [768]),
 ('decoder/block_000/layer_000/SelfAttention/v_slot_vr', [768]),
 ('decoder/block_000/layer_000/layer_norm/scale', [768]),

In [8]:
config = T5Config(
    vocab_size = 32128,
    n_positions=1024,
    d_ff = 3072,
    d_kv = 64,
    d_model = 768,
    dropout_rate = 0.1,
    inputs_length = 1024,
    num_heads = 12,
    num_layers = 12,
    decoder_start_token_id = 0,
    eos_token_id = 1,
    pad_token_id = 0)
print(config)
config.save_pretrained(out)

T5Config {
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "inputs_length": 1024,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 1024,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.21.2",
  "use_cache": true,
  "vocab_size": 32128
}



In [9]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-base-social-media/model.ckpt-1000000')

T5Model(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace

In [10]:
from transformers import CONFIG_NAME, WEIGHTS_NAME
CONFIG_NAME, WEIGHTS_NAME

('config.json', 'pytorch_model.bin')

In [12]:
import torch

torch.save(model.state_dict(), out + '/' + WEIGHTS_NAME)

In [13]:
from transformers import T5Config, T5Model, T5Tokenizer

In [14]:
# !wget https://f000.backblazeb2.com/file/malaya-model/bpe/sp10m.cased.ms-en-4k.model
# !wget https://f000.backblazeb2.com/file/malaya-model/bpe/sp10m.cased.ms-en.model

In [15]:
tokenizer = T5Tokenizer('sp10m.cased.ms-en.model')
tokenizer.save_pretrained(out)

('t5-base/tokenizer_config.json',
 't5-base/special_tokens_map.json',
 't5-base/spiece.model',
 't5-base/added_tokens.json')

In [16]:
tokenizer = T5Tokenizer.from_pretrained(f'./{out}', lower = False)
tokenizer.save_pretrained(out)

('t5-base/tokenizer_config.json',
 't5-base/special_tokens_map.json',
 't5-base/spiece.model',
 't5-base/added_tokens.json')

In [17]:
config = T5Config.from_pretrained(f'./{out}')
config

T5Config {
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "inputs_length": 1024,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 1024,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.21.2",
  "use_cache": true,
  "vocab_size": 32128
}

In [18]:
model = T5Model.from_pretrained(f'./{out}/pytorch_model.bin', config = config)
model

T5Model(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace

In [19]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [20]:
tokenizer = T5Tokenizer.from_pretrained(f'./{out}')
model = T5ForConditionalGeneration.from_pretrained(f'./{out}')
input_ids = tokenizer.encode('soalan: siapakah perdana menteri malaysia?', return_tensors = 'pt')
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

/home/husein/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'<pad> Mahathir Mohamad</s>'

In [21]:
%%time

input_ids = tokenizer.encode('soalan: gunung apa paling tinggi?', return_tensors = 'pt')
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

CPU times: user 937 ms, sys: 0 ns, total: 937 ms
Wall time: 81.5 ms


'<pad> Denali</s>'

In [24]:
%%time

input_ids = tokenizer.encode('terjemah Inggeris ke Melayu: Amid questions of Najib Razak serving time in hospital rather than a prison cell, to his active social media presence and loyalists calling for an early election, a jail term is not the end for the Malaysian former prime minister', return_tensors = 'pt')
outputs = model.generate(input_ids, max_length = 100)
tokenizer.decode(outputs[0])

CPU times: user 10.6 s, sys: 0 ns, total: 10.6 s
Wall time: 889 ms


'<pad> Amid questions of Najib Razak serving time in hospital rather than a prison cell, to his active media social presence and loyalists calling for an early election, a jail term is not end for the Malaysian former prime minister</s>'

In [25]:
model.push_to_hub('t5-base-bahasa-cased', organization='mesolitica',)
tokenizer.push_to_hub('t5-base-bahasa-cased', organization='mesolitica',)

/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:38: FutureWarning: Deprecated positional argument(s) used in 'create_repo': pass token='t5-base-bahasa-cased' as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:102: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.10. Pass `repo_id` instead.
  warnings.warn(
/home/husein/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:681: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/mesolitica/t5-base-bahasa-cased into local empty directory.


Upload file pytorch_model.bin:   0%|          | 32.0k/850M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mesolitica/t5-base-bahasa-cased
   3c95710..758c88a  main -> main



Upload file spiece.model:   4%|4         | 32.0k/784k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mesolitica/t5-base-bahasa-cased
   758c88a..e736fcb  main -> main



'https://huggingface.co/mesolitica/t5-base-bahasa-cased/commit/e736fcba485ef71f2752c961c5361ff6fefb2779'

In [23]:
!rm -rf t5-3x-super-tiny-4k-v3 t5-3x-super-tiny-4k-standard-bahasa-cased t5-3x-super-tiny-4k